<a href="https://colab.research.google.com/github/cekiorpes/big-data-challenge/blob/main/Level_1/Video_game_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-19 00:32:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-05-19 00:32:45 (5.94 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GameCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in review data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True, timestampFormat="yyyy-mm-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-01-31 00:08:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [5]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

1785997
1785886
1785886


In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



Transform data for review_id_table

In [8]:
#Create new DataFrame for review_id_table
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R101BFBMRAOEDT|   25979950|B002BRZ9G0|     795773015|2013-01-13 00:12:00|
|R101OYKK90GCD3|   26424123|B000ZK9QD2|     343904943|2009-01-23 00:10:00|
|R1025IN3G40UR0|   45793084|B00004TJCL|     754868066|2001-01-12 00:03:00|
|R102FS86G4B4N3|   46213053|B007FTE2VW|     265303108|2013-01-08 00:03:00|
|R102MY1UF3EV41|   29760851|B000035XCA|     480040881|2013-01-06 00:11:00|
|R102YIWZEIAXT9|   16385120|B000UQAUWW|     694091073|2007-01-29 00:11:00|
|R104KX3QSYZDMQ|   12772181|B002I0JBVY|     763179750|2013-01-23 00:01:00|
|R104LQ2H4P36H5|   46318135|B0050SXLQC|       8683886|2013-01-27 00:01:00|
|R105RLBQN5OIH7|   49538915|B0011E82F4|      66523065|2013-01-29 00:12:00|
|R1061FAGHHER0R|   18090895|B00HUWA45W|     753176240|2014-01-24 00:06:00|
|R106IO8YEZQ8LJ|   368811

In [9]:
#Write DataFrame to RDS
mode="append"
jdbc_url = "jdbc:postgresql://mypostgresdb.cojhn4lpsels.us-east-1.rds.amazonaws.com:5432/big_data_db"
config = {"user":"root",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [10]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Transform data for products table

In [11]:
#Create new DataFrame for products table
products = df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002BRZ9G0|New Super Mario B...|
|B000ZK9QD2|      Gears of War 2|
|B00004TJCL|Half-Life: Counte...|
|B007FTE2VW|SimCity - Limited...|
|B000035XCA|Terminator 2: Jud...|
|B000UQAUWW|Xbox 360 Console ...|
|B002I0JBVY|       Kinect Sports|
|B0050SXLQC|     Resident Evil 6|
|B0011E82F4|   Xbox 360 AV Cable|
|B00HUWA45W|KontrolFreek FPS ...|
|B003O6JKLC|Xbox 360 4GB Console|
|B000VTQ3LU|     PlayStation Eye|
|B00000J2W7|   Super Smash Bros.|
|B00005QB9P|Rogue Spear Black...|
|B00002STAE|NHL '95 [E] [Sega...|
|B000654X0M|  Timesplitters- PS2|
|B001KX504C|Gold's Gym Cardio...|
|B003RNRD4S| Wii Stand (RVL-017)|
|B003WF4826|  10 Minute Solution|
|B00NNU07RU|    Xbox 360 Console|
+----------+--------------------+
only showing top 20 rows



In [13]:
#Drop duplicates
products = products.dropDuplicates()

In [14]:
#Write DataFrame to RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Transform data for customers table

In [15]:
#Create new DataFrame for customers table
customers = df.select(["customer_id"])
customers.show(5)

+-----------+
|customer_id|
+-----------+
|   25979950|
|   26424123|
|   45793084|
|   46213053|
|   29760851|
+-----------+
only showing top 5 rows



In [16]:
#Count ratings by customer_id
customers = df.groupby("customer_id").agg({"customer_id":"count"})
customers.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   12536485|                 3|
|   12907964|                 1|
|   45223495|                 2|
|   42622247|                 4|
|   12889161|                 1|
+-----------+------------------+
only showing top 5 rows



In [17]:
#Rename count column
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12536485|             3|
|   12907964|             1|
|   45223495|             2|
|   42622247|             4|
|   12889161|             1|
+-----------+--------------+
only showing top 5 rows



In [18]:
#Write DataFrame to RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Transform data for vine_table

In [19]:
#Create new DataFrame for vine_table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R101BFBMRAOEDT|          3|            0|          1|   N|
|R101OYKK90GCD3|          5|            0|          1|   N|
|R1025IN3G40UR0|          4|           36|         38|   N|
|R102FS86G4B4N3|          1|            0|          0|   N|
|R102MY1UF3EV41|          5|            0|          0|   N|
|R102YIWZEIAXT9|          5|            2|          4|   N|
|R104KX3QSYZDMQ|          5|            0|          0|   N|
|R104LQ2H4P36H5|          5|            0|          0|   N|
|R105RLBQN5OIH7|          4|            0|          1|   N|
|R1061FAGHHER0R|          5|            0|          0|   N|
|R106IO8YEZQ8LJ|          1|            5|         11|   N|
|R106K7GK45VBML|          4|            1|          1|   N|
|R106YGJPHK3E9V|          5|            3|          6|   N|
|R108A9Q26SJIUW|          5|            

In [20]:
#Write DataFrame to RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)